In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
import torch
import numpy as np

class convexLinear(torch.nn.Module):
    """ Custom linear layer with positive weights and no bias """
    def __init__(self, size_in, size_out, ):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = torch.nn.Parameter(weights)

        # initialize weights
        torch.nn.init.kaiming_uniform_(self.weights, a=np.sqrt(5))

    def forward(self, x):
        w_times_x= torch.mm(x, torch.nn.functional.softplus(self.weights.t()))
        return w_times_x

class ICNN(torch.nn.Module):
    def __init__(self, n_input, n_hidden, n_output, dropout):
        super(ICNN, self).__init__()
        # Create Module dicts for the hidden and skip-connection layers
        self.layers = torch.nn.ModuleDict()
        self.skip_layers = torch.nn.ModuleDict()
        self.depth = len(n_hidden)
        self.dropout = dropout[0]
        self.p_dropout = dropout[1]

        self.layers[str(0)] = torch.nn.Linear(n_input, n_hidden[0]).float()
        # Create create NN with number of elements in n_hidden as depth
        for i in range(1, self.depth):
            self.layers[str(i)] = convexLinear(n_hidden[i-1], n_hidden[i]).float()
            self.skip_layers[str(i)] = torch.nn.Linear(n_input, n_hidden[i]).float()

        self.layers[str(self.depth)] = convexLinear(n_hidden[self.depth-1], n_output).float()
        self.skip_layers[str(self.depth)] = convexLinear(n_input, n_output).float()

    def forward(self, x):

        # Get F components
        F11 = x[:,0:1]
        F12 = x[:,1:2]
        F21 = x[:,2:3]
        F22 = x[:,3:4]

        # Compute right Cauchy green strain Tensor
        C11 = F11**2 + F21**2
        C12 = F11*F12 + F21*F22
        C21 = F11*F12 + F21*F22
        C22 = F12**2 + F22**2

        # Compute computeStrainInvariants
        I1 = C11 + C22 + 1.0
        I2 = C11 + C22 - C12*C21 + C11*C22
        I3 = C11*C22 - C12*C21

        # Apply transformation to invariants
        K1 = I1 * torch.pow(I3,-1/3) - 3.0
        K2 = (I1 + I3 - 1) * torch.pow(I3,-2/3) - 3.0
        J = torch.sqrt(I3)
        K3 = (J-1)**2

        # Concatenate feature
        x_input = torch.cat((K1,K2,K3),1).float()

        z = x_input.clone()
        z = self.layers[str(0)](z)
        for layer in range(1,self.depth):
            skip = self.skip_layers[str(layer)](x_input)
            z = self.layers[str(layer)](z)
            z += skip
            z = torch.nn.functional.softplus(z)     
            z = 1/12.*torch.square(z)
            if self.training:
                if self.dropout:
                    z = torch.nn.functional.dropout(z,p=self.p_dropout)
        y = self.layers[str(self.depth)](z) + self.skip_layers[str(self.depth)](x_input)
        return y

In [67]:

n_input = 3
n_output = 1
n_hidden = [64,64,64]
dropout = [True,0.2]
model = ICNN(n_input=n_input,
                n_hidden=n_hidden,
                n_output=n_output,
                dropout=dropout)

model.load_state_dict(torch.load('ArrudaBoyce_noise=high.pth'))
model.eval()

# Create dummy deformation gradients based on uniaxial tension
F=torch.zeros(50,4)
gamma=torch.linspace(0,0.5,50)
for a in range(50):
    F[a,0] = 1 + gamma[a]
    F[a,1] = 0
    F[a,2] = 0
    F[a,3] = 1

# Zero input deformation gradient + track gradients
F_0 = torch.zeros((1,4))
F_0[:,0] = 1
F_0[:,3] = 1

F11_0 = F_0[:,0:1]
F12_0 = F_0[:,1:2]
F21_0 = F_0[:,2:3]
F22_0 = F_0[:,3:4]

F11_0.requires_grad = True
F12_0.requires_grad = True
F21_0.requires_grad = True
F22_0.requires_grad = True


# Get components of F of Uniaxial tension + track gradients
F11 = F[:,0:1]
F12 = F[:,1:2]
F21 = F[:,2:3]
F22 = F[:,3:4]

F11.requires_grad = True
F12.requires_grad = True
F21.requires_grad = True
F22.requires_grad = True

# Predict strain energy (uncorrected)
W_NN = model(torch.cat((F11,F12,F21,F22),dim=1))

# Get dW_NN/DF components
dW_NN_dF11 = torch.autograd.grad(W_NN,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]
dW_NN_dF12 = torch.autograd.grad(W_NN,F12,torch.ones(F12.shape[0],1),create_graph=True)[0]
dW_NN_dF21 = torch.autograd.grad(W_NN,F21,torch.ones(F21.shape[0],1),create_graph=True)[0]
dW_NN_dF22 = torch.autograd.grad(W_NN,F22,torch.ones(F22.shape[0],1),create_graph=True)[0]

# Assemble first PK stress tensor as shape (-1,4)
P_NN = torch.cat((dW_NN_dF11,dW_NN_dF12,dW_NN_dF21,dW_NN_dF22),dim=1)

# Predict energy at zero deformation for energy correction
W_NN_0 = model(torch.cat((F11_0,F12_0,F21_0,F22_0),dim=1))

# Predict stress at zero deformation for stress correction
dW_NN_dF11_0 = torch.autograd.grad(W_NN_0,F11_0,torch.ones(F11_0.shape[0],1),create_graph=True)[0]
dW_NN_dF12_0 = torch.autograd.grad(W_NN_0,F12_0,torch.ones(F12_0.shape[0],1),create_graph=True)[0]
dW_NN_dF21_0 = torch.autograd.grad(W_NN_0,F21_0,torch.ones(F21_0.shape[0],1),create_graph=True)[0]
dW_NN_dF22_0 = torch.autograd.grad(W_NN_0,F22_0,torch.ones(F22_0.shape[0],1),create_graph=True)[0]

P_NN_0 = torch.cat((dW_NN_dF11_0,dW_NN_dF12_0,dW_NN_dF21_0,dW_NN_dF22_0),dim=1)

# Assemble stress correction term
P_cor = torch.zeros_like(P_NN)

P_cor[:,0:1] = F11*-P_NN_0[:,0:1] + F12*-P_NN_0[:,2:3]
P_cor[:,1:2] = F11*-P_NN_0[:,1:2] + F12*-P_NN_0[:,3:4]
P_cor[:,2:3] = F21*-P_NN_0[:,0:1] + F22*-P_NN_0[:,2:3]
P_cor[:,3:4] = F21*-P_NN_0[:,1:2] + F22*-P_NN_0[:,3:4]

# obtain final stress
P = P_NN + P_cor

/var/folders/y0/23jtbc011ldb7ngvjccl2w9sr08f79/T/ipykernel_44847/363596085.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ArrudaBoyce

In [66]:
P_NN_0.reshape(2,2) @ P_NN_0.reshape(2,2)

tensor([[0., 0.],
        [0., 0.]], grad_fn=<MmBackward0>)

In [ ]:
# Voigt_map = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.int32)
# delta = torch.eye(2)

# delta_H = torch.zeros(num_components, num_components, requires_grad=True)
# H = P_NN_0.reshape(2,2)
# for c in range(num_components):
#     row = []
#     for s in range(num_components):
#         row.append(delta[Voigt_map[c][0], Voigt_map[s][0]] * H[Voigt_map[s][1], Voigt_map[c][1]])
#     delta_H[c] = torch.tensor(row)

torch.Size([50, 1])

In [77]:
# Concatenate all components of F into a single tensor
F = torch.cat((F11, F12, F21, F22), dim=1)

# Predict strain energy (uncorrected)
W_NN = model(F)

# Compute the gradient of W_NN with respect to the entire F tensor
P_NN = torch.autograd.grad(W_NN, F, torch.ones_like(W_NN), create_graph=True)[0]

F_0 = torch.cat((F11_0,F12_0,F21_0,F22_0),dim=1)
W_NN_0 = model(F_0)
P_NN_0 = torch.autograd.grad(W_NN_0, F_0, torch.ones_like(W_NN_0), create_graph=True)[0].squeeze()

P_cor = torch.zeros_like(P_NN)

P_cor[:,0] = F[:,0]*-P_NN_0[0] + F[:,1]*-P_NN_0[2]
P_cor[:,1] = F[:,0]*-P_NN_0[1] + F[:,1]*-P_NN_0[3]
P_cor[:,2] = F[:,2]*-P_NN_0[0] + F[:,3]*-P_NN_0[2]
P_cor[:,3] = F[:,2]*-P_NN_0[1] + F[:,3]*-P_NN_0[3]

P = P_NN + P_cor

# Initialize a tensor to store the full Jacobian (second derivative)
batch_size, num_components = F.shape
jacobian_rows = []

# Compute the Jacobian row by row
for i in range(num_components):
    grad_output = torch.zeros_like(P)
    grad_output[:, i] = 1.0  # Select the i-th component
    jacobian_rows.append(torch.autograd.grad(P, F, grad_output, create_graph=True)[0])

dP = torch.stack(jacobian_rows, dim=1)  # Shape: (batch_size, num_components, num_components)

print("Jacobian shape:", dP.shape)

Jacobian shape: torch.Size([50, 4, 4])


In [74]:
P_NN_0.squeeze()

tensor([0., 0., 0., 0.], grad_fn=<SqueezeBackward0>)

In [72]:
P_NN_0[:,0:1].shape

torch.Size([1, 1])

In [ ]:
F[:,0].shape

torch.Size([50])

In [56]:
# Stack the rows to form the full Jacobian tensor
dP_NN = torch.stack(jacobian_rows, dim=1)  # Shape: (batch_size, num_components, num_components)

print("Jacobian shape:", dP_NN.shape)

Jacobian shape: torch.Size([50, 4, 4])


In [61]:
x = torch.randn(3)
x
torch.stack([x,x,x]).shape

torch.Size([3, 3])

In [45]:
dW_NN_dF.shape

torch.Size([50, 4])

In [42]:
P_NN

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0656, 0.0000, 0.0000, 0.0125],
        [0.1302, 0.0000, 0.0000, 0.0258],
        [0.1940, 0.0000, 0.0000, 0.0397],
        [0.2570, 0.0000, 0.0000, 0.0544],
        [0.3192, 0.0000, 0.0000, 0.0698],
        [0.3806, 0.0000, 0.0000, 0.0858],
        [0.4412, 0.0000, 0.0000, 0.1026],
        [0.5012, 0.0000, 0.0000, 0.1201],
        [0.5604, 0.0000, 0.0000, 0.1382],
        [0.6190, 0.0000, 0.0000, 0.1571],
        [0.6769, 0.0000, 0.0000, 0.1766],
        [0.7342, 0.0000, 0.0000, 0.1968],
        [0.7909, 0.0000, 0.0000, 0.2177],
        [0.8471, 0.0000, 0.0000, 0.2393],
        [0.9026, 0.0000, 0.0000, 0.2616],
        [0.9576, 0.0000, 0.0000, 0.2846],
        [1.0121, 0.0000, 0.0000, 0.3082],
        [1.0661, 0.0000, 0.0000, 0.3325],
        [1.1196, 0.0000, 0.0000, 0.3575],
        [1.1725, 0.0000, 0.0000, 0.3832],
        [1.2251, 0.0000, 0.0000, 0.4095],
        [1.2772, 0.0000, 0.0000, 0.4365],
        [1.3288, 0.0000, 0.0000, 0

In [37]:
P11 = torch.autograd.grad(W_NN, F11, torch.ones(F11.shape[0],1), create_graph=True)[0]
dP11 = torch.autograd.grad(P11, F11, torch.ones(F11.shape[0],1), create_graph=True)[0]

In [25]:
tmp = torch.autograd.grad(P_NN, F11, torch.ones_like(dW_NN_dF11), create_graph=True)[0]

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([50, 1]) and output[0] has a shape of torch.Size([50, 4]).

In [24]:
d2W_NN_dF11_2.shape

torch.Size([50, 1])

In [6]:
dW_NN_dF11_0.shape

torch.Size([1, 1])

In [7]:
P_NN_0.shape

torch.Size([1, 4])

In [21]:
dW_NN_dF11.shape

torch.Size([50, 1])

In [ ]:
dW_NN_dF11 = torch.autograd.grad(W_NN,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]

In [ ]:
dP_dF11 = torch.autograd.grad(W_NN,F11,torch.ones(F11.shape[0],1),create_graph=True)[0]

In [8]:
torch.cat((F11,F12,F21,F22),dim=1).shape

torch.Size([50, 4])

In [9]:
F_0

tensor([[1., 0., 0., 1.]])

In [10]:
F_0[:,3:4]

tensor([[1.]])

In [11]:
F_0[:,3]


tensor([1.])

In [12]:
F_0.shape

torch.Size([1, 4])

In [13]:
torch.cat((F11_0,F12_0,F21_0,F22_0),dim=1).shape

torch.Size([1, 4])

In [14]:
F_0.requires_grad = True

In [15]:
W_NN_0 = model(F_0)


In [16]:
torch.ones(F_0.shape[0],1)

tensor([[1.]])

In [17]:
F_0.shape

torch.Size([1, 4])

In [18]:
F_0.requires_grad = True

dW_NN = torch.autograd.grad(W_NN_0, F_0, torch.ones_like(F_0), create_graph=True)[0]
dW_NN

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([1, 4]) and output[0] has a shape of torch.Size([1, 1]).